In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

In this guided project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

### Overview of the data

A query to return information on the tables in the database.

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


A query that returns the first 5 rows of the 'facts' table.

In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

### Summary statistics

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few initial observations:

- There's a country with a population of 0
- There's a country with a population of more than 7bn

Let's see what these countries are.

### Exploring outliers

In [5]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MIN(population)
                            FROM facts
                        );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MAX(population)
                            FROM facts
                        );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems that the 'facts' table contains rows for Antarctica with a population of 0 and a row for the whole world with a population of 7256490011. Let's recompute the summary statistics, but now excluding the row for the whole world.

### Revisiting summary statistics

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
        MAX(population) AS max_pop,
        MIN(population_growth) AS min_pop_growth,
        MAX(population_growth) AS max_pop_growth
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


For when this database was compiled, the country with the largest population had almost 1.4bn people!

### Exploring average population and area

Let's look at average values for population and area, not including the row for the whole world.

In [8]:
%%sql
SELECT AVG(population) AS avg_population,
        AVG(area) AS avg_area
    FROM facts
    WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


The average population was around 32m people and the average area was around 555 thousand square kilometers.

### Finding densely populated counties

To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have:

- Above average values for population.
- Below average values for area.

In [9]:
%%sql
SELECT *
    FROM facts
    WHERE population > (SELECT AVG(population)
                           FROM facts
                           WHERE name <> 'World'
                       )
    AND area < (SELECT AVG(area)
                    FROM facts
                    WHERE name <> 'World'
                );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


The above tables contains counties that are densely populated, i.e. that have a population bigger than average that resides on an area smaller than average.

Next steps for to explore:

- What country has the most people? What country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries have more water than land?
- Which countries will add the most people to their population next year?
- Which countries have a higher death rate than birth rate?
- What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

### What country has the most people? What country has the highest growth rate?

In [10]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MAX(population)
                            FROM facts
                            WHERE name <> 'World'
                        );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


In [11]:
%%sql
SELECT *
    FROM facts
    WHERE population_growth == (SELECT MAX(population_growth)
                                FROM facts
                                WHERE name <> 'World'
                                );

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


China has the most people with almost 1.4bn and South Sudan has the higest population growth percentage at 4.02% annually.

### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [12]:
%%sql
SELECT name as Country, ROUND(CAST(area_water AS FLOAT) / area_land, 2) AS water_land_ratio
    FROM facts
    ORDER BY water_land_ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


Country,water_land_ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


These are the top 10 counties with the higest water to land ratios. We can also observe that British Indian Ocean Territory and Virgin Islands are the only countries where water area is larger than land area.

### Which countries will add the most people to their population next year?

In [13]:
%%sql
SELECT name, population, population_growth, 
       ROUND(CAST(population_growth * population as Float)/100) as population_added 
    FROM facts
    WHERE name <> 'World'
    ORDER BY population_added DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


name,population,population_growth,population_added
India,1251695584,1.22,15270686.0
China,1367485388,0.45,6153684.0
Nigeria,181562056,2.45,4448270.0
Pakistan,199085847,1.46,2906653.0
Ethiopia,99465819,2.89,2874562.0
Bangladesh,168957745,1.6,2703324.0
United States,321368864,0.78,2506677.0
Indonesia,255993674,0.92,2355142.0
"Congo, Democratic Republic of the",79375136,2.45,1944691.0
Philippines,100998376,1.61,1626074.0


The above countries will add the most to their population next year.

### Which countries have a higher death rate than birth rate?

In [14]:
%%sql
SELECT name AS Country, death_rate, birth_rate, ROUND (death_rate / birth_rate, 2) AS Ratio
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY Ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


Country,death_rate,birth_rate,Ratio
Bulgaria,14.44,8.92,1.62
Serbia,13.66,9.08,1.5
Latvia,14.31,10.0,1.43
Lithuania,14.27,10.1,1.41
Hungary,12.73,9.16,1.39
Monaco,9.24,6.65,1.39
Germany,11.42,8.47,1.35
Slovenia,11.37,8.42,1.35
Ukraine,14.46,10.72,1.35
Saint Pierre and Miquelon,9.72,7.42,1.31


Bulgaria, Serbia and Latvia have the highest ratio of deaths to births.

### What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

In [15]:
%%sql
SELECT name AS Country, population, area, ROUND (population / area, 2) AS Ratio
    FROM facts
    ORDER BY Ratio DESC
    LIMIT 10;

 * sqlite:///factbook.db
Done.


Country,population,area,Ratio
Macau,592731,28,21168.0
Monaco,30535,2,15267.0
Singapore,5674472,697,8141.0
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.0
Gibraltar,29258,6,4876.0
Bahrain,1346613,760,1771.0
Maldives,393253,298,1319.0
Malta,413965,316,1310.0
Bermuda,70196,54,1299.0
